<a id="1"></a>
# <p style="background-color:#f2efcb;font-family:arial;color:#7a2500;font-size:100%;text-align:center;border-radius:40px 40px; padding : 10px"> **IMPORTS AND SETUP**</p>

For this project, we have primarily used 3 libraries - 
 1. pyTorch, as the deep learning framework
 2. music21, to fetch the notes from the dataset.


In [ ]:
#Installing dependencies
!pip install music21
!apt-get install -y lilypond

In [5]:
# Importing helping libraries
import tensorflow as tf
import numpy as np 
import os
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [ ]:
# Dataset
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
# pyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data.dataloader import DataLoader
import torchvision
import torchvision.transforms as tt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<a id="2"></a>
# <p style="background-color:#f2efcb;font-family:arial;color:#7a2500;font-size:100%;text-align:center;border-radius:40px 40px; padding : 10px"> **LOADING AND PARSING DATA**</p>

For this project, I will be using MIDI files of classical piano music. The dataset includes various artists. I will be working with Frédéric Chopin's compositions. 
 
* First of all, I make a list of all the songs in the Chopin folder parsed as music21 stream.

* Then I will be creating a function to extract chords and notes out of the data creating a corpus.

**Laoding and parsing data**

In [ ]:
#Loading the list of chopin's midi files as stream 
filepath = "../input/classical-music-midi/mozart/"
#Getting midi files
all_midis = []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        print(i)
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)
        
# filepath = "../input/classical-music-midi/mozart/"
# #Getting midi files
# # all_midis= []
# for i in os.listdir(filepath):
#     if i.endswith(".mid"):
#         print(i)
#         tr = filepath+i
#         midi = converter.parse(tr)
#         all_midis.append(midi)

In [ ]:
print(all_midis[0])

In [ ]:
#Helping function        
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        print(j)
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
#                 print(element)
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
#         break

    return notes

# extract_notes(all_midis)
#Getting the list of notes as Corpus
# Corpus = extract_notes(all_midis)
# print("Total notes in all the Mozart midis in the dataset:", len(Corpus))

In [ ]:
Corpus[:200]

<a id="2"></a>
# <p style="background-color:#f2efcb;font-family:arial;color:#7a2500;font-size:100%;text-align:center;border-radius:40px 40px; padding : 10px"> **DATA EXPLORATION**</p>

In [ ]:
print("First fifty values in the Corpus:", Corpus[:50])

In [ ]:
#First Lets write some functions that we need to look into the data
def show(music):
    display(Image(str(music.write("lily.png"))))
    
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

# Melody_Snippet = chords_n_notes(Corpus[:100])
# show(Melody_Snippet)

**Playing the above sheet music** 

*As I could not play a midi file on the Kaggle interface, I have created a ".wav" filetype of the same outside of this code. I am using it to create an audio interface. Let us have a listen to the data corpus.* 

In [ ]:
#to play audio or corpus
print("Sample Audio From Data")
IPython.display.Audio("../input/music-generated-lstm/Corpus_Snippet.wav") 

**Examine all the notes in the Corpus** 

In [ ]:
#Creating a count dictionary
count_num = Counter(Corpus)
print("Total unique notes in the Corpus:", len(count_num))

In [ ]:
#Exploring the notes dictionary
Notes = list(count_num.keys())
Recurrence = list(count_num.values())
#Average recurrenc for a note in Corpus
def Average(lst):
    return sum(lst) / len(lst)
print("Average recurrenc for a note in Corpus:", Average(Recurrence))
print("Most frequent note in Corpus appeared:", max(Recurrence), "times")
print("Least frequent note in Corpus appeared:", min(Recurrence), "time")

Clearly, there are some very rare notes in the melody; some so rare that it was played only once in the whole data. This would create a lot of problems. (I did run into most of them while writing this piece)
To spare us the error reports, let us have a look at the frequency of the notes. 
And for simplicity, I shall be eliminating some of the least occurring notes. I am sure Chopin wouldn't mind me messing with his masterpiece for science or would he? Either way, I may never know!   

In [ ]:
# Plotting the distribution of Notes
plt.figure(figsize=(18,3),facecolor="#97BACB")
bins = np.arange(0,(max(Recurrence)), 50) 
plt.hist(Recurrence, bins=bins, color="#97BACB")
plt.axvline(x=100,color="#DBACC1")
plt.title("Frequency Distribution Of Notes In The Corpus")
plt.xlabel("Frequency Of Chords in Corpus")
plt.ylabel("Number Of Chords")
plt.show()

I have decided, I will be taking out the notes that were played less than 100 times. I mean, if Chopin liked them he would have played it a lot more often. So I create a list of rare notes in the next section. 

In [ ]:
#Getting a list of rare chords
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100:
        m =  key
        rare_note.append(m)
        
print("Total number of notes that occur less than 100 times:", len(rare_note))

In [ ]:
#Eleminating the rare notes
for element in Corpus:
    if element in rare_note:
        Corpus.remove(element)

print("Length of Corpus after elemination the rare notes:", len(Corpus))

<a id="2"></a>
# <p style="background-color:#f2efcb;font-family:arial;color:#7a2500;font-size:100%;text-align:center;border-radius:40px 40px; padding : 10px"> **DATA PREPROCESSING**</p>

Notes in music can be described as sound waves with specific combinations of frequency and wavelength. The names of these notes are stored in our database, and when we loaded the data, we used the music21 library from MIT to retrieve additional information such as frequency, wavelength, and duration for each note.


**This section will comprise of the follwoing tasks-**

**Creating a dictionary**: We are establishing a dictionary to establish a mapping between musical notes and their corresponding indices. In our dataset, the notes are represented as strings, but for the computer, they are merely symbols. Thus, we create a dictionary to associate each unique note in our dataset with a numerical value. This enables us to encode and decode information when utilizing the Recurrent Neural Network (RNN).

**Encoding and Splitting the corpus**: The next step involves encoding and dividing the dataset into smaller sequences of the same length. At this stage, the dataset consists of musical notes. We encode these notes and generate sequences with consistent lengths, containing both features and their corresponding targets. Each feature and target are represented by the mapped index from the dictionary, representing the unique characters.

**Assigning X and Y**: Following that, we resize and normalize the labels while applying one-hot encoding to the targets. This prepares the data to be fed into the RNN for training. However, prior to that, we need to construct the RNN model.

**Splitting Train and Seed datasets**: To generate music, we need to provide some initial input to the RNN. Therefore, we set aside a portion of the data as "seeds" for this purpose. While it would be possible to train the model using the entire dataset, I, as a non-musician, lack the ability to determine an appropriate input seed value.

**Creating a list of sorted unique characters**: This involves generating a list containing all the distinct characters found in the dataset. The characters are sorted in a particular order within the list.







In [ ]:
# Storing all the unique characters present in my corpus to bult a mapping dic. 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

**Encoding and Splitting the Corpus as Labels and Targets**

In [ ]:
Corpus[:50]

In [ ]:
#Splitting the Corpus in equal length of strings and output target
length = 40
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
    
L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

In [ ]:
features[:10]

In [ ]:
def to_bin(x):
    fin = ""
    a = bin(x)[2:]
    for _ in range(8-len(a)): fin+='0'
    fin+=a
    ans = []
    for ch in fin:ans.append(int(ch))
    return torch.tensor(ans)

to_bin(169)

In [ ]:
# reshape X and normalize
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(L_symb)
# one hot encode the output variable
# y = tensorflow.keras.utils.to_categorical(targets)
y = (torch.tensor(targets))
# y = np.float(y)

In [ ]:
X_tt = (np.reshape(features, (L_datapoints, length, 1)))
X_trans = torch.randn(X_tt.shape[0], X_tt.shape[1], 8)#torch.load('../input/trans-test/X_trans.pt')t

In [ ]:
for i in range(X_tt.shape[0]):
    for j in range(X_tt.shape[1]):
        X_trans[i][j] = to_bin(X_tt[i][j][0])
#         print(X_trans[i][j])

In [ ]:
y.shape

In [ ]:
#Taking out a subset of data to be used as seed
#Change X and X_trans depending upon model to use 
# X_train, X_seed, y_train, y_seed = train_test_split(X_trans, y, test_size=0.2, random_state=42)

train_ds = torch.load('../input/trans-test/train_ds.pt')#[]
test_ds = torch.load('../input/trans-test/test_ds.pt')#[]

# for i in range(len(X_train)):
#     train_ds.append([X_train[i], y_train[i]])
    
# for i in range(len(X_seed)):
#     test_ds.append([X_seed[i], y_seed[i]])
    
train_dl = DataLoader(train_ds, batch_size = 256, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = 256, shuffle = True)

In [ ]:
for x,y in train_dl:
    print(x.shape)
    print(y.shape)
    break

In [ ]:
torch.save(train_ds, 'train_ds.pt')
torch.save(test_ds, 'test_ds.pt')

**Splitting Train and Seed datasets**

<a id="2"></a>
# <p style="background-color:#f2efcb;font-family:arial;color:#7a2500;font-size:100%;text-align:center;border-radius:40px 40px; padding : 10px"> **MODEL BUILDING**</p>

In [ ]:
#Initialising the Model
model = Sequential()
#Adding layers
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
#Compiling the model for training  
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)


In [ ]:
model = nn.Sequential(
        nn.LSTM(1, 512, 2, batch_first=True, dropout = 0.1),
        nn.LSTM(1, 256, 1, batch_first=True, dropout = 0.1)
    
        )

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.LSTM(1, 512, 1, batch_first=True, dropout=0.1)
        self.l2 = nn.LSTM(512, 256, 1, batch_first=True, dropout=0.1)
        self.d = nn.Linear(256, 266)
        
    def forward(self, x):
        out, (hn, _) = self.l1(x.float())
        out, (hn, _) = self.l2(out)
        output = self.d(hn)
        
        output = output.reshape(output.shape[1], output.shape[2])
        return output

In [ ]:
class RNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.LSTM(1, 128, 1, batch_first=True, dropout=0.1)
        self.l2 = nn.LSTM(128, 256, 1, batch_first=True, dropout=0.1)
        self.p = nn.Linear(256,256)
        self.d = nn.Linear(256, 266)
        self.drop = nn.Dropout(p=0.1)
        
    def forward(self, x):
        out, (hn, _) = self.l1(x.float())
#         print(out.shape, hn.shape)
        out = self.drop(out)
        out, (hn, _) = self.l2(out)
        hn = self.drop(hn)
        output = self.p(hn)
        output = self.drop(output)
        output = self.d(output)
        
        output = output.reshape(output.shape[1], output.shape[2])
        return output

In [ ]:
class RNN3(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.LSTM(1, 64, 1, batch_first=True, bidirectional = True)
        self.l2 = nn.LSTM(128, 256, 1, batch_first=True)
        self.p = nn.Linear(256,256)
        self.d = nn.Linear(256, 269)
        self.drop = nn.Dropout(p=0.1)
        
    def forward(self, x):
        out, (hn, _) = self.l1(x.float())
#         print(out.shape, hn.shape)
        out = self.drop(out)
        out, (hn, _) = self.l2(out)
#         print(out.shape, hn.shape)
#         hn = hn.reshape(-1, 1, 512)
#         print(hn.shape)
        hn = self.drop(hn)
        output = self.p(hn)
        output = self.drop(output)
        output = self.d(output)
#         print(output.shape)
        
        output = output.reshape(output.shape[1], output.shape[2])
        return output

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model
from tensorflow import keras

# from attention import Attention

model_input = Input(shape=(40, 8))
x = LSTM(64, return_sequences=True)(model_input)
x = Attention(units=32)(x)
x = Dense(169)(x)
model = Model(model_input, x)


opt = tf.keras.optimizers.Adam(learning_rate=1e-3, name="Adam")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.000001)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.losses.CategoricalCrossentropy()])
model.summary()

# opt = keras.optimizers.Adam(learning_rate=1e-4)
# model.compile(loss='categorical_crossentropy', optimizer=opt)
# model.summary()

In [ ]:
model.fit(X_train.numpy(), y_train.numpy(), validation_split=0.2, epochs=30, batch_size=256,
          verbose=1, callbacks=[reduce_lr, early_stopping])

In [ ]:
class trans(nn.Module):
    def __init__(self):
        super().__init__()
        # 256, 40, 32
        self.el1 = nn.TransformerEncoderLayer(d_model=16, nhead=4)
        self.lstm = nn.LSTM(8, 16, 1, batch_first=True)
        self.drop = nn.Dropout(p=0.1)
        # 320
        self.mlp = nn.Sequential(nn.Linear(640, 256), nn.Linear(256, 169))
        
    def forward(self, x):
        out, (_, _)  = self.lstm(x)
        out = self.drop(out)
        out = self.el1(out)
        out = self.drop(out)
        out = nn.Flatten()(out)
#         print(out.shape)
        output = self.mlp(out)
#         print(output.shape)
#         output = output.reshape(output.shape[1], output.shape[2])
        return output

In [ ]:
model = trans().to(device)
x = torch.randn(256,40,8).to(device)
y = model(x)
y.shape

In [ ]:
model = RNN().to(device)

In [ ]:
model

In [ ]:
for x,y in train_dl:
    print((y[0][0]))
    break

In [ ]:
inp = torch.randn(32, 40, 8).to(device)
y = model(inp)
y.shape

In [ ]:
opt = torch.optim.Adamax(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    model.train()
    for batch_data,(x,y) in enumerate(dataloader):
        x,y = x.to(device), y.to(device)

        # Compute prediction error
        pred = model(x)
#         print(pred.shape, y.shape)
        loss = loss_fn(pred,y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_data % 100 == 0:
            loss, current = loss.item(), batch_data * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
# model = Summary_CNN()
# opt = torch.optim.Adam(model.parameters(), lr=0.001)

# loss_fn = F.mse_loss

# for epoch in range(50):
#   train(test_dl, model, loss_fn=loss_fn, optimizer=opt)

In [ ]:
# model = trans().to(device)

opt = torch.optim.Adamax(model.parameters(), lr = 1e-4)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(30):
    print(epoch+1)
    train(train_dl, model, loss_fn=loss_fn, optimizer=opt)

In [ ]:
#Model's Summary               
model.summary()

In [ ]:
#Training the Model
history = model.fit(X_train, y_train, batch_size=256, epochs=200)

<a id="2"></a>
# <p style="background-color:#f2efcb;font-family:arial;color:#7a2500;font-size:100%;text-align:center;border-radius:40px 40px; padding : 10px"> **EVALUATING MODELS**</p>

In [ ]:
#Plotting the learnings 
history_df = pd.DataFrame(history.history)
fig = plt.figure(figsize=(15,4), facecolor="#97BACB")
fig.suptitle("Learning Plot of Model for Loss")
pl=sns.lineplot(data=history_df["loss"],color="#444160")
pl.set(ylabel ="Training Loss")
pl.set(xlabel ="Epochs")

**Generating the Melody**

A function to obtain the generated music

In [ ]:
from random import choices

def Malody_Generator(Note_count):
    seed = X_seed[np.random.randint(0,len(X_seed)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(100):
        seed = seed.reshape(1,length,1)
        seedt = torch.tensor(seed).to(device)
        prediction = model(seedt)
        prediction = F.softmax(prediction)
#         print(prediction.shape)
        index = choices(range(269), weights = prediction[0], k=1)[0]
#         print(index)
#         print(prediction.shape)
#         prediction = np.log(prediction) / 1.0 #diversity
#         exp_preds = np.exp(prediction)
#         prediction = exp_preds / np.sum(exp_preds)
#         index = prediction
        index_N = index/ float(L_symb)   
        Notes_Generated.append(index)
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi


#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(100)
show(Melody)

This sure looks like music! To check if it sounds like music we have to listen to the MIDI file. Playing midi is crumblesome. I have saved and converted a few generated melodies to ".wav" format outside of this notebook. So let us have a listen. 

**Melody Generated Sample 1**

In [ ]:
#To save the generated melody
Melody.write('midi','Melody_Generated_khichdi.mid')
#to play audio or corpus
# IPython.display.Audio("../input/music-generated-lstm/Melody_Generated 2.wav")

**Melody Generated Sample 2**

In [ ]:
#to play audio or corpus
IPython.display.Audio("../input/music-generated-lstm/Melody_Generated_1.wav")